# Programación Lineal

En esta práctica el objetivo es resolver una situación particular del Transportation Problem.

Nuestra empresa tiene n depósitos y m locales de venta. Para simplificar, vamos a asumir que solo hay tres tipos de productos a comercializar.
Por cada uno de los productos tendremos la información del stock disponible en cada depósito. Además, tendremos la demanda de cada local.
Por último, por cada tipo de producto y por cada par depósito-local, tendremos el costo de enviar una unidad de mercadería.

El objetivo es encontrar la forma óptima de transporte desde los depósitos a los locales, cumpliendo las restricciones de stock y demanda. Se entiende por óptima a la forma que menor costo asociado tenga.

In [ ]:
!pip install ortools

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 55.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 31.5 MB/s eta 0:00:00
  Attempting uninstall: protobuf
    Found existing installation: protobuf 3.20.3
    Uninstalling protobuf-3.20.3:
      Successfully uninstalled protobuf-3.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow-metadata 1.14.0 requires protobuf<4.21,>=3.20.3, but you have protobuf 4.24.3 which is incompatible.


In [ ]:
from ortools.linear_solver import pywraplp
import random

### Datos

In [ ]:
# Definimos cantidades
n_depositos = 2
n_locales = 3
n_productos = 3

In [ ]:
# generamos un diccionario donde la key es la combinacion de deposito y producto
# asignamos un stock aleatorio para cada deposito y producto
stock = {}
for deposito in range(n_depositos):
    for producto in range(n_productos):
        stock[(f"(d{deposito}, p{producto})")] = random.randint(200, 300)

# stock

In [ ]:
# generamos un diccionario donde la key es la combinacion de local y producto
# asignamos una demanda aleatorio para cada local y producto
demanda = {}
for local in range(n_locales):
    for producto in range(n_productos):
        demanda[(f"(l{local}, p{producto})")] = random.randint(50, 100)

# demanda

In [ ]:
# generamos un diccionario donde la key es la combinacion de deposito, local y producto
# asignamos un costo aleatorio para el traslado desde cada deposito hacia local por producto
costo = {}
for deposito in range(n_depositos):
    for local in range(n_locales):
        for producto in range(n_productos):
            costo[(f"(d{deposito}, l{local}, p{producto})")] = random.uniform(1, 2)

# costo

### Solver

In [ ]:
# Creamos el solver de Google OR-Tools
solver = pywraplp.Solver.CreateSolver('SCIP')

### Variables
Definimos las variables de decisión que contendrán la cantidad de unidades de cada producto que se envían desde un depósito a un local.


In [ ]:
cantidad_productos = {}
for dep in range(n_depositos):
    for loc in range(n_locales):
        for prod in range(n_productos):
            # solver.NumVar(0, solver.infinity() crea la variable que puede tomar valores no negativos desde cero hasta Infinito.
            cantidad_productos[dep, loc, prod] = solver.NumVar(0, solver.infinity(), f"Cantidad_Productos_{dep}_{loc}_{prod}")

cantidad_productos

{(0, 0, 0): Cantidad_Productos_0_0_0,
 (0, 0, 1): Cantidad_Productos_0_0_1,
 (0, 0, 2): Cantidad_Productos_0_0_2,
 (0, 1, 0): Cantidad_Productos_0_1_0,
 (0, 1, 1): Cantidad_Productos_0_1_1,
 (0, 1, 2): Cantidad_Productos_0_1_2,
 (0, 2, 0): Cantidad_Productos_0_2_0,
 (0, 2, 1): Cantidad_Productos_0_2_1,
 (0, 2, 2): Cantidad_Productos_0_2_2,
 (1, 0, 0): Cantidad_Productos_1_0_0,
 (1, 0, 1): Cantidad_Productos_1_0_1,
 (1, 0, 2): Cantidad_Productos_1_0_2,
 (1, 1, 0): Cantidad_Productos_1_1_0,
 (1, 1, 1): Cantidad_Productos_1_1_1,
 (1, 1, 2): Cantidad_Productos_1_1_2,
 (1, 2, 0): Cantidad_Productos_1_2_0,
 (1, 2, 1): Cantidad_Productos_1_2_1,
 (1, 2, 2): Cantidad_Productos_1_2_2}

### Función objetivo
Buscamos miniminzar el costo de transportar las cantidades de producto necesarias desde los depósitos hacia los locales cumpliendo las restricciones pertinentes.

In [ ]:
solver.Minimize(
    solver.Sum(
        cantidad_productos[dep, loc, prod] * costo[f"(d{dep}, l{loc}, p{prod})"]
        for dep in range(n_depositos)
        for loc in range(n_locales)
        for prod in range(n_productos)
    )
)

### Restricciones


*   **Restricción de stock**: La cantidad de cada producto que se envía desde cada depósito tiene que ser menor o igual al stock con que cuenta.

*   **Restricción de demanda**: La cantidad de cada producto que recibe cada local tiene que ser mayor o igual a su demanda.



In [ ]:
# Restricciones de stock
# Para cada depósito, evaluamos cada producto y sumamos las cantidades enviadas a todos los locales, siendo esta suma menor o igual a su stock.
for dep in range(n_depositos):
    for prod in range(n_productos):
        solver.Add(
            solver.Sum(cantidad_productos[dep, loc, prod] for loc in range(n_locales)) <= stock[f"(d{dep}, p{prod})"]
        )


In [ ]:
# Restricciones de demanda
# Para cada local, evaluamos cada producto y sumamos la cantidad recibida de todos los depósitos, debiendo ser por lo menos igual a la cantidad demandada.
for loc in range(n_locales):
    for prod in range(n_productos):
        solver.Add(
            solver.Sum(cantidad_productos[dep, loc, prod] for dep in range(n_depositos)) >= demanda[f"(l{loc}, p{prod})"]
        )


In [ ]:
# for loc in range(n_locales):
#     for prod in range(n_productos):
#         solver.Add(
#             solver.Sum(cantidad_productos[dep, loc, prod] for dep in range(n_depositos)) == demanda[f"(l{loc}, p{prod})"]
#         )

### Resolución

In [ ]:
result = solver.Solve()

if result == solver.OPTIMAL:
    print("La solución es óptima.")
elif result == solver.FEASIBLE:
    print("Se encontró una solución factible, pero no necesariamente óptima.")
elif result == solver.INFEASIBLE:
    print("No se encontró una solución factible.")
elif result == solver.UNBOUNDED:
    print("El problema es no acotado.")
else:
    print("No se pudo resolver el problema por alguna razón desconocida.")

La solución es óptima.


In [ ]:
# Imprimir resultados
print ("Valor objetivo óptimo:", solver.Objective().Value())
print("")
print("Flujo óptimo:")
for dep in range(n_depositos):
    for loc in range(n_locales):
        for prod in range(n_productos):
            cantidad = cantidad_productos[dep, loc, prod].solution_value()
            print(f"Enviar {cantidad} unidades del Producto {prod+1} desde el Depósito {dep+1} al Local {loc+1}")


Valor objetivo óptimo: 711.4006905801429

Flujo óptimo:
Enviar 0.0 unidades del Producto 1 desde el Depósito 1 al Local 1
Enviar 0.0 unidades del Producto 2 desde el Depósito 1 al Local 1
Enviar 54.0 unidades del Producto 3 desde el Depósito 1 al Local 1
Enviar 0.0 unidades del Producto 1 desde el Depósito 1 al Local 2
Enviar 0.0 unidades del Producto 2 desde el Depósito 1 al Local 2
Enviar 0.0 unidades del Producto 3 desde el Depósito 1 al Local 2
Enviar 0.0 unidades del Producto 1 desde el Depósito 1 al Local 3
Enviar 0.0 unidades del Producto 2 desde el Depósito 1 al Local 3
Enviar 0.0 unidades del Producto 3 desde el Depósito 1 al Local 3
Enviar 59.0 unidades del Producto 1 desde el Depósito 2 al Local 1
Enviar 67.0 unidades del Producto 2 desde el Depósito 2 al Local 1
Enviar 0.0 unidades del Producto 3 desde el Depósito 2 al Local 1
Enviar 59.0 unidades del Producto 1 desde el Depósito 2 al Local 2
Enviar 50.0 unidades del Producto 2 desde el Depósito 2 al Local 2
Enviar 99.0 uni

# Definimos la clase y funciones necesarias para replicar el modelo con distintos inputs.

In [ ]:
import random
from ortools.linear_solver import pywraplp
import time

class TransportProblemSolver:
    def __init__(self, n_depositos, n_locales, n_productos):
        self.n_depositos = n_depositos
        self.n_locales = n_locales
        self.n_productos = n_productos

        # Variables del modelo
        self.stock = {}
        self.demanda = {}
        self.costo = {}
        self.cantidad_productos = {}
        self.solver = None

    # Generamos nuestros datos.
    def generate_data(self):
        stock = {}
        for deposito in range(self.n_depositos):
            for producto in range(self.n_productos):
                stock[(f"(d{deposito}, p{producto})")] = random.randint(200, 300)

        demanda = {}
        for local in range(self.n_locales):
            for producto in range(self.n_productos):
                demanda[(f"(l{local}, p{producto})")] = random.randint(50, 100)

        costo = {}
        for deposito in range(self.n_depositos):
            for local in range(self.n_locales):
                for producto in range(self.n_productos):
                    costo[(f"(d{deposito}, l{local}, p{producto})")] = random.uniform(1, 2)

        return stock, demanda, costo

    # Definimos el solver a usar.
    def solve(self):
        solver = pywraplp.Solver.CreateSolver('SCIP')
        if not solver:
            return None

        stock, demanda, costo = self.generate_data()

        # Definimos las variables de decisión que contendrán la cantidad de unidades de cada producto que se envían desde un depósito a un local.
        cantidad_productos = {}
        for dep in range(self.n_depositos):
            for loc in range(self.n_locales):
                for prod in range(self.n_productos):
                    # solver.NumVar(0, solver.infinity() crea la variable que puede tomar valores no negativos desde cero hasta Infinito.
                    cantidad_productos[dep, loc, prod] = solver.NumVar(0, solver.infinity(), f"Cantidad_Productos_{dep}_{loc}_{prod}")

        # Función objetivo.
        solver.Minimize(
            solver.Sum(
                cantidad_productos[dep, loc, prod] * costo[f"(d{dep}, l{loc}, p{prod})"]
                for dep in range(self.n_depositos)
                for loc in range(self.n_locales)
                for prod in range(self.n_productos)
            )
        )

        # Restricción de stock.
        for dep in range(self.n_depositos):
            for prod in range(self.n_productos):
                solver.Add(
                    solver.Sum(cantidad_productos[dep, loc, prod] for loc in range(self.n_locales)) <= stock[f"(d{dep}, p{prod})"]
                )

        # Restricción de demanda.
        for loc in range(self.n_locales):
            for prod in range(self.n_productos):
                solver.Add(
                    solver.Sum(cantidad_productos[dep, loc, prod] for dep in range(self.n_depositos)) >= demanda[f"(l{loc}, p{prod})"]
                )

        result = solver.Solve()

        return solver, result, cantidad_productos



In [ ]:
# Definimos la función que nos permité ejecutar nuestro modelo a partir de la clase definida anteriormente.
# Capturamos e imprimimos la información que consideramos relevante.
def modelo(n_depositos, n_locales):
    n_productos = 3

    start_time = time.time()
    solver_instance = TransportProblemSolver(n_depositos, n_locales, n_productos)
    solver, result, cantidad_productos = solver_instance.solve()
    end_time = time.time()
    duration = end_time - start_time

    print(f"Tiempo de resolución: {round(duration,2)} segundos")
    print("")

    if result == solver.OPTIMAL:
        print("La solución es óptima.")
    elif result == solver.FEASIBLE:
        print("Se encontró una solución factible, pero no necesariamente óptima.")
    elif result == solver.INFEASIBLE:
        print("No se encontró una solución factible.")
    elif result == solver.UNBOUNDED:
        print("El problema es no acotado.")
    else:
        print("No se pudo resolver el problema por alguna razón desconocida.")

    print("")
    print("Valor objetivo óptimo:", solver.Objective().Value())
    print("")
    # print("Flujo óptimo:")
    # for dep in range(n_depositos):
    #     for loc in range(n_locales):
    #         for prod in range(n_productos):
    #             cantidad = cantidad_productos[dep, loc, prod].solution_value()
    #             print(f"Enviar {cantidad} unidades del Producto {prod+1} desde el Depósito {dep+1} al Local {loc+1}")



## Testeamos resultados con distintas cantidades de depósitos y locales.

In [ ]:
modelo(n_depositos=2, n_locales=3)

Tiempo de resolución: 0.01 segundos

La solución es óptima.

Valor objetivo óptimo: 841.8951269747513



In [ ]:
modelo(n_depositos=100, n_locales=50)

Tiempo de resolución: 0.56 segundos

La solución es óptima.

Valor objetivo óptimo: 11300.724543138265



In [ ]:
modelo(n_depositos=100, n_locales=200)

Tiempo de resolución: 2.97 segundos

La solución es óptima.

Valor objetivo óptimo: 45233.27773775124



In [ ]:
modelo(n_depositos=500, n_locales=500)

Tiempo de resolución: 58.09 segundos

La solución es óptima.

Valor objetivo óptimo: 113437.27708130445



### El problema presentado está íntimamente relacionado con el Problema de Asignación. Este problema también se puede resolver con un modelo de Programación Lineal, pero también existen algoritmos procedurales para resolver directamente el problema. Por ejemplo, el Algoritmo Húngaro es un algoritmo que sirve para resolver el Problema de Asignación en un tiempo polinomial. ¿Por qué podría seguir siendo interesante el uso de un modelo de Programación Lineal para atacar el problema?

Los modelos de Programación Lineal nos permiten modelar el problema que queremos resolver, no hacemos foco en cómo armar nuestra solución sino en qué solución debemos armar de manera tal de alcanzar cierta versatilidad para no reformular nuestro algoritmo ante pequeños cambios en nuestro problema.

La Programación Lineal nos aporta facilidad en la formulación así como también flexibilidad ante variaciones en nuestro problema. Si bien los algoritmos procedurales como el Algoritmo Húngaro son altamente eficientes para resolver el Problema de Asignación y pueden ser preferibles en situaciones en las que se necesita una solución específica, los modelos de Programación Lineal permiten abordar problemas complejos obteniendo soluciones globales óptimas.